In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
model= ChatOpenAI(model_name="gpt-3.5-turbo")

### Lets go Step by Step

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I am Vikas")])

AIMessage(content='Hello Vikas! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-36166270-a8c1-4a23-b9fa-fa200c64042b-0')

* Now lets check if it remembers my name

In [6]:

model.invoke([HumanMessage(content="Whats my name?")])

AIMessage(content="I'm sorry, I am not able to know your name as I am an AI assistant.", response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 11, 'total_tokens': 30}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-07a39054-1cfe-4a52-9560-4c7458b59f57-0')

* It didn't remember the name
    * Now we will save some predefined chat and ask the model if it remembers old chat

In [9]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Vikas"),
        AIMessage(content="Hello Vikas! How can i help you?"),
        HumanMessage(content="What's my name")
        
    ]
)

AIMessage(content='Your name is Vikas.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 35, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8c60a6c2-10c2-4e66-b74a-20c1ac862f50-0')

### Now it is remembering the name
* So Chatbot has to have previous messages to remember what we have told
#### Lets Streamline this

* We will store our chat in data store and use it for future interactions by retrieving them and pass them into chain as part of input.
* We will manage this with session ids

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [11]:
store= {}
## Get session history would give you the chat history of the session or it will store it if it is new
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= ChatMessageHistory()
    return store[session_id]

## Managing chat: Where storing and appending new chat happens here
with_message_history= RunnableWithMessageHistory(model, get_session_history)

* We will create config now that we have to pass into runnables everytime. This config consists of session id

In [12]:
config= {"configurable": {"session_id": "vikas"}}

In [22]:
response= with_message_history.invoke(
    [
        HumanMessage(content="Hello, I'm Vikas")
    ],
    config=config
)


Parent run 892271da-2437-4afb-adba-c7ce312b768c not found for run fab5071c-a8be-4d4a-8d73-ff3d0b772431. Treating as a root run.


In [15]:
response.content

'Nice to meet you Vikas! How can I help you today?'

* Now lets ask my name again

In [16]:
response= with_message_history.invoke(
    [
        HumanMessage(content="What is my name")
    ], config= config
)
response.content

Parent run 75bbe2e1-2a7e-4747-8c6d-c77a7c48b50e not found for run 4495bb6e-5a09-4c2e-a357-253c709a7cdb. Treating as a root run.


'Your name is Vikas.'

## Cool! Now the chat remembers history

* Lets change the session id and check

In [25]:
config_2= {"configurable": {"session_id": "vicky"}}
response= with_message_history.invoke(
    [
        HumanMessage(content="What is my name?")
    ], config= config_2
)
response.content

Parent run c7247948-8de3-4589-8fae-81de60d39fe8 not found for run fceed298-f397-4ffb-8fae-9717de453b84. Treating as a root run.


"I'm sorry, I am not able to know your name as we are communicating textually."

* Now lets goback and ask

In [26]:

respose= with_message_history.invoke(
    [
        HumanMessage(content="What is my name?")
        
    ],
    config= config
)
response.content

Parent run 6e37ce29-18e1-45ba-91db-18bd780737d5 not found for run fce58b8c-73cf-4149-b5e9-ce13f134f20d. Treating as a root run.


"I'm sorry, I am not able to know your name as we are communicating textually."

## This is very basic layer of the chat history
### Lets dig more into it and understand
* We will find a way to attach history to prompt template as well

In [102]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "you are a helpful assistant. Answer all the questions to the best of your ability."
            template
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain= prompt | model

In [103]:
response= chain.invoke(
    {
        "messages": [
            HumanMessage(content="Hi! I'm vikas")
        ]
    }
)
response.content


'Hello Vikas! How can I assist you today?'

### Now lets add message history to the chat
* We will create Runnablewithhistory but instead of model we will pass chain as parameter

In [29]:
with_message_history= RunnableWithMessageHistory(chain, get_session_history)
config= {"configurable": {"session_id":"vikas"}}

In [107]:
template

" You are certificate generator expert.\nYou collect details from input to create certificate mentioned below along with the list of details required from user\n\n\nBelow are the types of certificates:\n\n\n\nEmployment Resignation:\nThis certificate generates resignation details of employee. If language is German then Formular id= ARB_AUS else Formular id= ARB_AUS_EN and generates certificate provided below information:\n1. Employee Id: \n2. Bv Number:\n3. Effective Date (YYYY-MM-DD):\n4. Language (English or German): \n\nIf all the necessary values are provided then covert the result in url format with prefix https://certificates/certificate name/certificate details seperated by '/'.\n\n\nEmployment details:\nThis certificate generates employment details of employee requested. If language is German then Formular id= ARB_ESS else Formular id= ARB_ESS_EN and generates certificate provided below information:\n1. Employee Id: \n2. Bv Number:\n3. Effective Date (YYYY-MM-DD):\n4. Language 

In [111]:
response= with_message_history.invoke(
    [
        HumanMessage(content="Can you create certificates?")
    ],
    config= config
)

# response.content
response

Parent run da6fe4b4-431b-48fb-8b40-00eab2f41c2c not found for run fb4910cd-3279-43c8-b25f-07eb2fbdefd2. Treating as a root run.


'\nAI: Yes, I can create various types of certificates. Please let me know which type of certificate you would like me to generate.'

In [31]:
response= with_message_history.invoke(
    [
        HumanMessage(content="What is my name")
    ],
    config= config
)

response.content

Parent run bdb0eb84-36ba-45fa-84b9-36162eedf8a7 not found for run 545fe9da-1477-4cf8-95d3-7764fd9b0c1e. Treating as a root run.


'Your name is Vikas.'

## Super! Lets add few other parameters such as respond in different language

In [61]:
prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all question tp best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain= prompt| model

In [68]:
response= chain.invoke(
    {"messages":[HumanMessage(content="Hello, I'm Vikas")], "language": "Spanish"
        }
)
response.content

'¡Hola, Vikas! ¿En qué puedo ayudarte hoy?'

In [70]:
## Create runnable char history
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [71]:
config= {"configurable": {"session_id":"vikas"}}

In [72]:
response= with_message_history.invoke(
    {"messages":[HumanMessage(content="Hello, I'm Vikas")], "language": "Spanish"
        }, config=config
)
response.content

Parent run 547af9f6-73aa-4af9-ae1c-200332e6226a not found for run f280e049-b033-441b-97e5-b3159912d62c. Treating as a root run.


'Nice to meet you Vikas! How can I help you today?'

In [75]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="What is my name?")
        ], "language":"Spanish"
    }, config=config
)
response.content

Parent run 07f1a320-d82e-4776-a531-093084cb325e not found for run 37aab3e0-700e-450e-a558-0ce0f05a930c. Treating as a root run.


'Your name is Vikas.'

### Managing Chat history
* We can add chat history like above. But what if storing goes on for all the chat. 
    * Memory issue
    * Overflowing of the context window of LLM
* So We will add a step to limit the size of the messages that we are passing in
* This has to be done Before the prompt template, but After loading messages from message history.
* We can do this by creating a function to select k most recent messages.

In [82]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
    return messages[-k:]

chain= (
    RunnablePassthrough.assign(messages= lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [83]:
messages = [
    HumanMessage(content="hi! I'm Vikas"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like butterscotch ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [84]:
response =  chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my name")],
        "language":"English"
    }
)
response.content

"I'm sorry, I don't have access to that information."

### This is because The limit is 10 and the entered messages is 11th so it left the first message

In [85]:
response =  chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my Fav ice Cream")],
        "language":"English"
    }
)
response.content

'Your favorite ice cream flavor is butterscotch.'

* The response was within the chat

#### Lets Wrap with message history with session id

In [86]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config= {"configurable": {"session_id": "vikas"}}

In [87]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="What is my name?")
        ], "language":"english"
    }, config= config
)
response.content

Parent run 0b83f644-f02d-4b8b-882d-0b2ee771f719 not found for run daf71cf2-f1d1-4abf-a7b8-47ecd721a421. Treating as a root run.


'Your name is Vikas.'

In [88]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="What is my favourite ice cream?")
        ], "language":"english"
    }, config= config
)
response.content

Parent run c16e2839-e503-4e07-bf11-23c4e903ec78 not found for run 4178130e-1340-4655-9291-94c1a8ca8a3b. Treating as a root run.


"I'm sorry, I don't have that information. Could you please tell me what your favorite ice cream flavor is?"

In [89]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="It is butter scotch?")
        ], "language":"english"
    }, config= config
)
response.content

Parent run a51f764b-a2b9-4694-8c62-a2cb63cb48ac not found for run 2a4d82f7-d246-4cd0-9d69-2f114c9a19a2. Treating as a root run.


"That's great! Butter scotch is a delicious choice for an ice cream flavor."

In [90]:
response= with_message_history.invoke(
    {
        "messages": [
            HumanMessage(content="Can you tell me now, what is my favourite ice cream?")
        ], "language":"english"
    }, config= config
)
response.content

Parent run 8a0f7d6e-7d91-44f8-9164-6812c2b0d5d4 not found for run f1437241-7578-4049-9839-de20cbef020e. Treating as a root run.


'Your favorite ice cream flavor is butter scotch.'

In [92]:
from pprint import pprint
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    pprint(r.content)

Parent run 96cb9944-b889-4a86-8e2e-cc36f38dd289 not found for run 4797d4f1-0d3c-4005-a6f3-9fdfd14a6f1a. Treating as a root run.


''
'Hi'
' Todd'
'!'
' Sure'
','
' here'
"'s"
' a'
' joke'
' for'
' you'
':'
' Why'
' did'
' the'
' math'
' book'
' look'
' sad'
'?'
' Because'
' it'
' had'
' too'
' many'
' problems'
'.'
' 😄'
''


In [120]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAI
### Packages which handle Chat history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
##Chat prompt Template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo')
## Define LLLM congifuration
template = """
You are creating a Certificate of Employment for an employee. Please provide the following details:

Employment Resignation:
This certificate generates resignation details of employee. If language is German then Formular id= ARB_AUS else Formular id= ARB_AUS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

If all the necessary values are provided then covert the result in url format with prefix https://certificates/certificate name/certificate details seperated by '/'.


Employment details:
This certificate generates employment details of employee requested. If language is German then Formular id= ARB_ESS else Formular id= ARB_ESS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

Please provide each detail one by one in the given order. If you have any questions or need clarification, feel free to ask!

"""
# Now we can override it and set it to "Friend"
# from langchain_core.prompts.prompt import PromptTemplate

prompt =    ChatPromptTemplate.from_messages( [
        (
            "system",
            template
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
### Lets Create Chain
chain= prompt | llm

### Chat message history
## Function that stores chat based on session id
store= {}
## Get session history would give you the chat history of the session or it will store it if it is new
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= ChatMessageHistory()
    return store[session_id]

with_message_history= RunnableWithMessageHistory(
    chain, get_session_history,
       
)
## Create session id configuration
config= {"configurable": {"session_id":"vikas"}}


In [121]:
input_text= "Hi i am Vikas"

In [122]:
response= with_message_history.invoke(
    {
        "message": HumanMessage(content= input_text)
    }, config= config
)
print(response.content)

Parent run b576c209-ae9c-4b43-8ee7-0a5848b5a04c not found for run 54b4dc74-0f48-4b21-a542-281ab1c91a6e. Treating as a root run.


Hello Vikas! How can I assist you today?


In [ ]:

template = """
You are creating a Certificate of Employment for an employee. Please provide the following details:

Employment Resignation:
This certificate generates resignation details of employee. If language is German then Formular id= ARB_AUS else Formular id= ARB_AUS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

If all the necessary values are provided then covert the result in url format with prefix https://certificates/certificate name/certificate details seperated by '/'.


Employment details:
This certificate generates employment details of employee requested. If language is German then Formular id= ARB_ESS else Formular id= ARB_ESS_EN and generates certificate provided below information:
1. Employee Id: 
2. Bv Number:
3. Effective Date (YYYY-MM-DD):
4. Language (English or German): 

Please provide each detail one by one in the given order. If you have any questions or need clarification, feel free to ask!
"""

chat_prompt_template = ChatPromptTemplate.from_template(template)
